# Lab 9 - Named Entity Recognition -- Starting at 18:05

In this lab, we will explore models for Named Entity Recognition (as discussed in week 8).

NER systems can take many approaches, but the most common is based on sequence classification. That includes older statistical models like HMM, MEMM, and CRF, as well as newer neural models.


***Note***: Sample code in this labs require specific versions of some packages to be installed, which may cause dependency confilicts with some of the pre-installed packages in Colab. Thus, you may see some error messages indicating the dependency conflicts upon installation. These errors can be ignored and will not affect running of our sample codes since we will not use those conflicted packages.

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install -U 'scikit-learn<0.24'
!pip install urllib3==1.25.11

import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 44.6 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Installing build dependencies ... error
error: subprocess-exited-with-error

× pip subprocess to install build dependencies did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.0/128.0 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.15
    Uninstalling urllib3-1.26.1

In [ ]:
# read IOB tagged NER dataset into a dataframe
df = pd.read_csv('https://drive.google.com/uc?id=1UPHg29593FCCNqec6RyYitjfoyY90UWN', encoding = "ISO-8859-1")
df.head(10)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
5,NaN,through,IN,O
6,NaN,London,NNP,B-geo
7,NaN,to,TO,O
8,NaN,protest,VB,O
9,NaN,the,DT,O


Let's see the set of labels in this data:

In [ ]:
set(df['Tag'].tolist())

{'B-art',
 'B-eve',
 'B-geo',
 'B-gpe',
 'B-nat',
 'B-org',
 'B-per',
 'B-tim',
 'I-art',
 'I-eve',
 'I-geo',
 'I-gpe',
 'I-nat',
 'I-org',
 'I-per',
 'I-tim',
 'O'}

Uou can see the different types of entities: 
* geo = Geographical Entity
* org = Organization
* per = Person
* gpe = Geopolitical Entity
* tim = Time indicator
* art = Artifact
* eve = Event
* nat = Natural Phenomenon

## Data Preprocessing
First, this dataset has some NaN values that we'll replace with 0 for convenience.

We can also count how many unique sentences, words, and tags we have:
- 47959 sentences
- 35172 unique words / types
- 17 tags

In [ ]:
df = df.fillna(method='ffill')
df['Sentence #'].nunique(), df.Word.nunique(), df.Tag.nunique() # nunique = the number of unique ones

(47959, 35172, 17)

In [ ]:
df.groupby('Tag').size().reset_index(name='counts')

,Tag,counts
0,B-art,402
1,B-eve,308
2,B-geo,37644
3,B-gpe,15870
4,B-nat,201
5,B-org,20143
6,B-per,16990
7,B-tim,20333
8,I-art,297
9,I-eve,253


We will now train a CRF model for named entity recognition using sklearn-crfsuite on our dataset. We are going to use a fork of the library because of some issues running the code otherwise.

In [ ]:
!pip install git+https://github.com/MeMartijn/updated-sklearn-crfsuite.git#egg=sklearn_crfsuite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/MeMartijn/updated-sklearn-crfsuite.git to /tmp/pip-install-0tt97abn/sklearn-crfsuite_414bc65cc57e4c59bf7803f0ce014d0b
  Running command git clone --filter=blob:none --quiet https://github.com/MeMartijn/updated-sklearn-crfsuite.git /tmp/pip-install-0tt97abn/sklearn-crfsuite_414bc65cc57e4c59bf7803f0ce014d0b
  Resolved https://github.com/MeMartijn/updated-sklearn-crfsuite.git to commit 675038761b4405f04691a83339d04903790e2b95
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 14.0 MB/s eta 0:00:00
  Created wheel for sklearn_crfsuite: filename=sklearn_crfsuite-0.3.6-py2.py3-none-any.whl size=10888 sha256=1fb01353c66800a48306a300e2b872a5f552366e3e37e5bfb31d49d6e024d35a
  Stored in directory: /tmp/pip-ephem-wheel-cache-iyyu0j7k/wheels/0b/bc/07/bd75a6f5fa2bf2ea05a5aad8d9ac66d2b5aab93dfd4e1a89de
Successfully built skle

##Conditional random fields (CRF)

This [post](http://blog.echen.me/2012/01/03/introduction-to-conditional-random-fields/) introduce CRF clearly.

This [post](https://dax-cdn.cdn.appdomain.cloud/dax-groningen-meaning-bank-modified/1.0.2/data-preview/Part%202%20-%20Named%20Entity%20Recognition.html) may help you in understaind the coding part.

In [ ]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from collections import Counter

In [ ]:
# lambda argument : expression
## apply the expression to the argument, and return the result

# Add 10 to argument a, and return the result: 
def addfun(a):
    b = a + 10
    return b

x = lambda a : a + 10
print(x(5))
print(addfun(5))



## this is a function!
# lamba function and map()
agg_func = lambda s: [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(), 
                                                    s['POS'].values.tolist(), 
                                                    s['Tag'].values.tolist())]
# argument = s; expression = [...]
## i.e.,
def def_agg_func(s):
    # print(type(s)) # <class 'pandas.core.frame.DataFrame'>
    # print(s) # each s contains tokens from ONE sentence -- i.e., all items in s share the same sentence id
    wpt = []
    for w,p,t in zip(s['Word'].values.tolist(),
                s['POS'].values.tolist(),
                s['Tag'].values.tolist()):
        wpt.append((w,p,t))
    return wpt
    

15


In [ ]:
#Retrieving sentences with their POS and tags.
class SentenceGetter(object):
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(), 
                                                           s['POS'].values.tolist(), 
                                                           s['Tag'].values.tolist())]
        self.grouped = self.data.groupby('Sentence #').apply(agg_func)
        
        data_grouped = self.data.groupby('Sentence #')
        print(type(data_grouped))
        print(data_grouped.head(10))

        groupp = data_grouped.apply(def_agg_func)
        
        print('------------------------------')
        print(self.grouped[10])
        print(groupp[10]) #<zip object at 0x7fa6c0d57380>
        
        self.sentences = [s for s in self.grouped]
        print(self.sentences[10])
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None
getter = SentenceGetter(df)
sentences = getter.sentences

<class 'pandas.core.groupby.generic.DataFrameGroupBy'>
              Sentence #           Word  POS Tag
0            Sentence: 1      Thousands  NNS   O
1            Sentence: 1             of   IN   O
2            Sentence: 1  demonstrators  NNS   O
3            Sentence: 1           have  VBP   O
4            Sentence: 1        marched  VBN   O
...                  ...            ...  ...  ..
1048570  Sentence: 47959           they  PRP   O
1048571  Sentence: 47959      responded  VBD   O
1048572  Sentence: 47959             to   TO   O
1048573  Sentence: 47959            the   DT   O
1048574  Sentence: 47959         attack   NN   O

[474199 rows x 4 columns]
------------------------------
[('In', 'IN', 'O'), ('Beirut', 'NNP', 'B-geo'), (',', ',', 'O'), ('a', 'DT', 'O'), ('string', 'NN', 'O'), ('of', 'IN', 'O'), ('officials', 'NNS', 'O'), ('voiced', 'VBD', 'O'), ('their', 'PRP$', 'O'), ('anger', 'NN', 'O'), (',', ',', 'O'), ('while', 'IN', 'O'), ('at', 'IN', 'O'), ('the', 'DT', 'O'),

We extract more features (word parts, simplified POS tags, lower/title/upper flags, features of nearby words) and convert them to sklearn-crfsuite format — each sentence should be converted to a list of dicts. The following code was taken from [the official sklearn-crfsuites site](https://sklearn-crfsuite.readthedocs.io/en/latest/tutorial.html).

In [ ]:
print(sentences[0]) # word-pos-tag

[('Thousands', 'NNS', 'O'), ('of', 'IN', 'O'), ('demonstrators', 'NNS', 'O'), ('have', 'VBP', 'O'), ('marched', 'VBN', 'O'), ('through', 'IN', 'O'), ('London', 'NNP', 'B-geo'), ('to', 'TO', 'O'), ('protest', 'VB', 'O'), ('the', 'DT', 'O'), ('war', 'NN', 'O'), ('in', 'IN', 'O'), ('Iraq', 'NNP', 'B-geo'), ('and', 'CC', 'O'), ('demand', 'VB', 'O'), ('the', 'DT', 'O'), ('withdrawal', 'NN', 'O'), ('of', 'IN', 'O'), ('British', 'JJ', 'B-gpe'), ('troops', 'NNS', 'O'), ('from', 'IN', 'O'), ('that', 'DT', 'O'), ('country', 'NN', 'O'), ('.', '.', 'O')]


The word2features function will generate additional features on each of our tokens. Because we would like our model to see the adjacent words of each of our tokens as it trains on our data, we structure word2features so that it takes in a full sentence (represented as a list of lists) and the word to generate features on (represented as an index of the supplied sentence). This makes it easier to reference adjacent words as we generate features on them by simply incrementing or decrementing the supplied index.

We also create the sentence2features and sentence2labels functions which assist in converting our data at the sentence level. sentence2features generates the feature dicts for each word in a sentence by calling word2features and sentence2labels generates a list of entity labels for each sentence.

In [ ]:
def word2features(sent, i):
    """
    Generates a feature dictionary for a word using the word's position in a sentence
    ref: https://sklearn-crfsuite.readthedocs.io/en/latest/tutorial.html

    :param sentence: a sentence stored as a list of lists
    :param i: the index of the word in sentence to generate features for
    :returns: feature dictionary for the supplied word
    """
    
    word = sent[i][0] 
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    
    if i > 0: 
        word1 = sent[i-1][0] # the word in previous timestep
        postag1 = sent[i-1][1] # the postag in previous timestep
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True # if i==0, meaning that it is the begining of sentences

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True # if i==len(sent)-1, meaning that it is the end of sentences

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))] # i is the index of the word in its sentence

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [ ]:
sent2features(sentences[0])[0] ## features of the first token in the first sentence

{'bias': 1.0,
 'word.lower()': 'thousands',
 'word[-3:]': 'nds',
 'word[-2:]': 'ds',
 'word.isupper()': False,
 'word.istitle()': True,
 'word.isdigit()': False,
 'postag': 'NNS',
 'postag[:2]': 'NN',
 'BOS': True,
 '+1:word.lower()': 'of',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:postag': 'IN',
 '+1:postag[:2]': 'IN'}

In [ ]:
# data splitting for training and testing

X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

Next we will train the model, this could take 3-5 minutes.

In [ ]:
# train a CRF model for named entity recognition using sklearn-crfsuite 
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', # Gradient descent using the L-BFGS method
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)

crf.fit(X_train, y_train)

CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    max_iterations=100)

In assignment 2, we do span-based evaluation. Here, we will do a simple tag-based evluation (ie., is the tag for each word correct?).

First, we are going to remove the “O” tag (outside). We do this because it is very common and will dominate our calculation otherwise.

In [ ]:
y = df.Tag.values
classes = np.unique(y)
classes = classes.tolist()
print(classes)
print('------------')
classes.pop()
print(classes)
# classes

['B-art', 'B-eve', 'B-geo', 'B-gpe', 'B-nat', 'B-org', 'B-per', 'B-tim', 'I-art', 'I-eve', 'I-geo', 'I-gpe', 'I-nat', 'I-org', 'I-per', 'I-tim', 'O']
------------
['B-art', 'B-eve', 'B-geo', 'B-gpe', 'B-nat', 'B-org', 'B-per', 'B-tim', 'I-art', 'I-eve', 'I-geo', 'I-gpe', 'I-nat', 'I-org', 'I-per', 'I-tim']


In [ ]:
#evaluation
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(y_test, y_pred, labels=classes))

              precision    recall  f1-score   support

       B-art       0.45      0.12      0.19       143
       B-eve       0.59      0.42      0.49       106
       B-geo       0.86      0.91      0.88     12447
       B-gpe       0.97      0.94      0.95      5284
       B-nat       0.82      0.42      0.56        78
       B-org       0.80      0.73      0.76      6615
       B-per       0.85      0.82      0.84      5652
       B-tim       0.93      0.88      0.90      6856
       I-art       0.11      0.03      0.05       105
       I-eve       0.38      0.22      0.28        93
       I-geo       0.82      0.81      0.81      2520
       I-gpe       0.91      0.62      0.74        69
       I-nat       1.00      0.43      0.61        23
       I-org       0.82      0.80      0.81      5597
       I-per       0.85      0.90      0.87      5674
       I-tim       0.84      0.74      0.79      2207

   micro avg       0.86      0.85      0.85     53469
   macro avg       0.75   

The following shows what our classifier learned. It gives high scores to transitions from the beginning of an entity type (e.g., B-nat) followed by an inside token of the same type (e.g., I-nat), but transitions from the beginning of one entity type to the inside of a different entity type are given negative scores.

In [ ]:
def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))
print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))
print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
B-nat  -> I-nat   6.934503
I-art  -> I-art   6.260215
B-art  -> I-art   5.881224
I-eve  -> I-eve   5.847777
B-eve  -> I-eve   5.586673
I-tim  -> I-tim   5.204188
I-org  -> I-org   4.782243
I-gpe  -> I-gpe   4.699609
B-tim  -> I-tim   4.636703
B-org  -> I-org   4.282602
O      -> O       3.813956
B-per  -> I-per   3.698815
I-geo  -> I-geo   3.685166
B-gpe  -> I-gpe   3.597376
B-geo  -> I-geo   3.516476
I-per  -> I-per   3.245863
I-nat  -> I-nat   2.954009
I-geo  -> B-art   1.973397
O      -> B-tim   1.748999
O      -> B-per   1.620428

Top unlikely transitions:
I-org  -> I-geo   -4.259782
I-org  -> I-per   -4.327937
B-geo  -> B-geo   -4.426926
B-per  -> I-org   -4.427218
B-geo  -> I-gpe   -4.435073
B-per  -> I-geo   -4.466408
B-tim  -> B-tim   -4.518613
B-org  -> I-geo   -4.575173
B-geo  -> I-per   -4.793920
B-org  -> I-per   -5.036090
B-geo  -> I-org   -5.070524
B-gpe  -> I-geo   -5.210003
B-gpe  -> I-org   -5.287803
B-gpe  -> B-gpe   -5.607401
O      -> I-per  

#Bi-LSTM CRF 
Here, we introduce a pytorch model for NER based on a Bi-LSTM CRF from the Pytorch official documentation: https://pytorch.org/tutorials/beginner/nlp/advanced_tutorial.html. 

## Example

In [ ]:
# Author: Robert Guthrie

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

### Helper functions to make the code more readable.

In [ ]:
def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()

def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

### Create model

In [ ]:
class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2),
                torch.randn(2, 1, self.hidden_dim // 2))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

#### nn.Module class

In [1]:
import torch
import torch.nn as nn

In [19]:
class BiLSTM_test(nn.Module):
    def __init__(self,aa=1):
        super(BiLSTM_test, self).__init__()
        ## The super call delegates the function call to the parent class, which is nn.Module in your case.
        ## This is actually needed to initialize the nn.Module properly. Check [https://docs.python.org/2/library/functions.html#super] for more information.
        # nn.Module.__init__(self) ## this line is just another way to initialize 
        print('__init__aa',aa)
    def forward(self,bb):
        print('bb',bb)
    def forward_test(self,cc):
        print('cc',cc)

In [20]:
aa = 1
bb = 2
cc = 3
model = BiLSTM_test()
forward_output = model(bb)

__init__aa 1
bb 2


### Run training

In [ ]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 5
HIDDEN_DIM = 4

# Make up some training data
training_data = [(
    "the wall street journal reported today that apple corporation made money".split(),
    "B I I I O O O B I O O".split()
), (
    "georgia tech is a university in georgia".split(),
    "B I O O O O B".split()
)]

word_to_ix = {}
for sentence, tags in training_data:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

tag_to_ix = {"B": 0, "I": 1, "O": 2, START_TAG: 3, STOP_TAG: 4}

model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

# Check predictions before training
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    precheck_tags = torch.tensor([tag_to_ix[t] for t in training_data[0][1]], dtype=torch.long)
    print(model(precheck_sent))

# Make sure prepare_sequence from earlier in the LSTM section is loaded
for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

# Check predictions after training
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    print(model(precheck_sent))
# We got it!

(tensor(2.6907), [1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1])
(tensor(20.4906), [0, 1, 1, 1, 2, 2, 2, 0, 1, 2, 2])
